In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from pandas import Index
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import request
import pickle

In [2]:
def signal_process(table,event_var='PM25_Marseille_Longchamp',event_threshold = 40, duration = 6,var_name_list='air pollution'):
    """Find the spike in pm25 and plot a sliding window for google trend """

    index_list=np.where(table[event_var]>event_threshold)[0]
    
    # add first element
    shrink_index_list = [index_list[0]]
    date_set ={table.date[index_list[0]]}
    
    # exclude for the same day
    for cnt in range(1,len(index_list)):
        if table.date[index_list[cnt]] in date_set:
            pass
        else:
            shrink_index_list.append(index_list[cnt].copy())
            date_set.add(table.date[index_list[cnt]])
    
    bind_list = []
    for start_index in shrink_index_list:
        this_sequence = table.loc[range(start_index-2,start_index+duration),
                                  var_name_list]
        start_date = table.datetime[start_index]
        tuples = list(zip([start_date]*(duration+2),range(-2,duration)))

        this_sequence.index = pd.MultiIndex.from_tuples(tuples,names=['event_date_index','hour_index'])
    #         this_sequence = (this_sequence- this_sequence.mean())/this_sequence.std()

        bind_list.append(this_sequence)
    new_table=pd.concat(bind_list)     
    
    return new_table

In [3]:
def fetching_trend(airdata, kw_list,location_code='FR-U',timezone = 0):
    input_table = airdata
    newest_datetime   = max(input_table['datetime'])
    earliest_datetime = min(input_table['datetime'])
    pytrends = TrendReq(hl='En-US', tz=timezone)
    print("fectching trend ")
    ggtrend_list =[]
    for keyword in kw_list:
        
        this_list = [keyword]
        try: 
            topic_code = pytrends.suggestions(keyword)[0]['mid']
            this_list.append(topic_code)
            print("keyword {} is encoded as {}".format(keyword,topic_code))
        except IndexError:
            pass
        print(this_list)
        ggtrend = pytrends.get_historical_interest(this_list, 
                                 year_start=earliest_datetime.year,
                                          month_start=earliest_datetime.month, 
                                          day_start=earliest_datetime.days_in_month, 
                                          hour_start=earliest_datetime.hour, 
                                 year_end=newest_datetime.year,
                                          month_end=newest_datetime.month, 
                                          day_end=newest_datetime.days_in_month, 
                                          hour_end=newest_datetime.hour, 
                                 cat=0, # all categorical
                                          geo=location_code,  # Provence-Alpes-Côte d'Azur
                                          gprop='',  # no specific group # image #hastag etc
                                          sleep=0)
        print("save temp file {}".format(keyword))
        with open("../stage/{}/{}.p".format(location_code,keyword),'bw') as f:
            pickle.dump(ggtrend,f)
            
        if len(this_list)==2:
            ggtrend.columns= Index([keyword, keyword+" topic", 'isPartial'], dtype='object')
        
        
        
        ggtrend_list.append(ggtrend)
    print("trend is fectched")
    return ggtrend_list


In [4]:
# loading table 

In [5]:
Limoges_table = pd.read_csv("../stage/limoges.csv").drop(columns=['Year','Month','Day','UTC Hour'])
Limoges_table.columns=Index(['PM25_Limoges', 'PM10_Limoges', 'datetime'], dtype='object')

In [7]:
Limoges_table.head()

,PM25_Limoges,PM10_Limoges,datetime
0,21.6,0.50,2013-01-01 00:00:00
1,15.7,0.63,2013-01-01 01:00:00
2,13.5,0.66,2013-01-01 02:00:00
3,11.0,0.68,2013-01-01 03:00:00
4,9.6,0.67,2013-01-01 04:00:00


In [8]:
parca_table = Limoges_table

In [9]:
full_date_time = pd.date_range(parca_table[['datetime']].min()[0],parca_table[['datetime']].max()[0],freq='H')
full_time_set = set(map(lambda mytime: mytime.strftime("%Y-%m-%d %H:%M:%S"),full_date_time.to_series())) 
data_time_set = set(map(lambda timestamp: str(timestamp),parca_table['datetime']))
    # pad na values
if len(full_time_set) != len(data_time_set):
    pad_time = full_time_set -data_time_set
    pad_table = pd.DataFrame()
    pad_table['datetime']=[datetime.strptime(mytime,"%Y-%m-%d %H:%M:%S") for mytime in pad_time]
    for column in parca_table.columns:
        if column !="datetime":
            pad_table[column]=None
    parca_table = pd.concat([parca_table,pad_table],ignore_index=True)

In [10]:
# fetching google trend

In [11]:
KW_LIST= [
#           'difficulté à respirer',
#           'asthme',
#           'banane',
#           'intoxication alimentaire',
#           'difficulté pour respirer',
#           'sensation d’étouffement',
#           'dyspnée',
#           'la diarrhée',
#           'spasmophilie',
#           'nuage de pollution',
#           'saleté de l’air',
#           'particules nocives',
#           'respiration',
#           'difficultés respiratoires',
#           'maladies respiratoires',
          'pollution de l’air',
#           'pollution atmosphérique',
#           'pollution',
#           'Masque', #mask
#           "Purificateur d'air", #air purifier
          
         ]

In [12]:
LOCATION_CODE = 'FR-B'  # ISO 3166 code	FR-B  for Nouvelle-Aquitaine mytime.strftime("%Y-%m-%d %H:%M:%S")

In [13]:
def conv(datestring):
    try :
        val = datetime.strptime(datestring ,'%Y-%m-%d %H:%M:%S')
    except TypeError:
        val = datestring
    return val

In [14]:
parca_table['datetime'] = parca_table['datetime'].apply(conv)

In [ ]:
ggtrend_list = fetching_trend(parca_table,kw_list=KW_LIST,location_code=LOCATION_CODE,timezone = 0)

fectching trend 
keyword pollution de l’air is encoded as /m/0dc7h
['pollution de l’air', '/m/0dc7h']
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google ret

The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 500.


In [ ]:
ggtrend=pd.concat(ggtrend_list,axis=1)

In [ ]:
ggtrend.reset_index(inplace=True)

In [ ]:
merged_t=parca_table.merge(ggtrend,left_on='datetime',right_on='date')
kw_str  ="_".join(KW_LIST)
merged_t.to_csv("../stage/parca_{}_{}.csv".format(kw_str,parca_table.columns[1]))

In [ ]:
signal_table=signal_process(merged_t,event_threshold = 35,duration = 12,var_name_list=list(ggtrend.columns)+['PM25_Marseille_Longchamp'])

In [ ]:
signal_table.groupby("hour_index").mean().plot()

In [ ]:
signal_table.to_csv("../output/air_parca_threshold_35_colsum_duration_12H.csv")